## Feature Engineering

**Your Python Jupyter notebook should be configured for >8GB of memory.**

In this series of tutorials, we will build a recommender system for fashion items. It will consist of two models: a *retrieval model* and a *ranking model*. The idea is that the retrieval model should be able to quickly generate a small subset of candidate items from a large collection of items. This comes at the cost of granularity, which is why we also train a ranking model that can afford to use more features than the retrieval model.

### Data

We will use data from the [H&M Personalized Fashion Recommendations](https://www.kaggle.com/competitions/h-and-m-personalized-fashion-recommendations) Kaggle competition.

<!-- https://www.kaggle.com/competitions/h-and-m-personalized-fashion-recommendations/data

For this challenge you are given the purchase history of customers across time, along with supporting metadata. Your challenge is to predict what articles each customer will purchase in the 7-day period immediately after the training data ends. Customer who did not make any purchase during that time are excluded from the scoring. -->

The full dataset contains images of all products, but here we will simply use the tabular data. We have three data sources:
- `articles.csv`: info about fashion items.
- `customers.csv`: info about users.
- `transactions_train.csv`: info about transactions.

You can use the *hopsworks* library to download these files locally, assuming that they are stored in your cluster. In this example, we have saved them to the `Resources` directory.

In [1]:
import hopsworks

project = hopsworks.login()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://hopsworks0.logicalclocks.com/p/119


In [2]:
import os

# download datasets
dataset_api = project.get_dataset_api()

data_dir = "data/"

if not os.path.exists(data_dir):
    os.mkdir(data_dir)

for file in ["articles.parquet", "customers.parquet", "transactions_train.parquet"]:
    dataset_api.download(f"Resources/{file}", local_path=data_dir, overwrite=True)

Downloading: 0.000%|          | 0/6445685 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/167680276 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/778112758 elapsed<00:00 remaining<?

In [3]:
import pandas as pd

# read articles data

data_dir = "data/"
articles_df = pd.read_parquet(data_dir + "articles.parquet")
articles_df["article_id"] = articles_df["article_id"].astype(str)
articles_df.head(3)

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.


In [4]:
articles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105542 entries, 0 to 105541
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   article_id                    105542 non-null  object
 1   product_code                  105542 non-null  int64 
 2   prod_name                     105542 non-null  object
 3   product_type_no               105542 non-null  int64 
 4   product_type_name             105542 non-null  object
 5   product_group_name            105542 non-null  object
 6   graphical_appearance_no       105542 non-null  int64 
 7   graphical_appearance_name     105542 non-null  object
 8   colour_group_code             105542 non-null  int64 
 9   colour_group_name             105542 non-null  object
 10  perceived_colour_value_id     105542 non-null  int64 
 11  perceived_colour_value_name   105542 non-null  object
 12  perceived_colour_master_id    105542 non-null  int64 
 13 

In [5]:
# read customers data

import pandas as pd

customers_df = pd.read_parquet(data_dir + "customers.parquet")
customers_df.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN,NaN,ACTIVE,NONE,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...


In [6]:
#customers_df.drop(customers_df.tail(int(1356119/4)*3).index, inplace = True) # reduce size of the data frame if needed
customers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1371980 entries, 0 to 1371979
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   customer_id             1371980 non-null  object 
 1   FN                      476930 non-null   float64
 2   Active                  464404 non-null   float64
 3   club_member_status      1365918 non-null  object 
 4   fashion_news_frequency  1355971 non-null  object 
 5   age                     1356119 non-null  float64
 6   postal_code             1371980 non-null  object 
dtypes: float64(3), object(4)
memory usage: 73.3+ MB


In [7]:
# read transactions data

trans_df = pd.read_parquet(data_dir + "transactions_train.parquet")
trans_df.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


In [8]:
# trans_df.drop(trans_df.index[:-3788324], inplace=True) # reduce size of the data frame if needed
trans_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   t_dat             object 
 1   customer_id       object 
 2   article_id        int64  
 3   price             float64
 4   sales_channel_id  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 1.2+ GB


In [9]:
trans_df["article_id"] = trans_df["article_id"].astype(str)
trans_df['t_dat'] = trans_df['t_dat'].apply(lambda x: pd.to_datetime(x))
trans_df.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


In [10]:
trans_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 5 columns):
 #   Column            Dtype         
---  ------            -----         
 0   t_dat             datetime64[ns]
 1   customer_id       object        
 2   article_id        object        
 3   price             float64       
 4   sales_channel_id  int64         
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 1.2+ GB


In [11]:
print(f"There are {len(trans_df):,} transactions in total.")

There are 31,788,324 transactions in total.


We can see that we have a large dataset. For the sake of the tutorial, we will use a small subset of this dataset, which we generate by sampling 25'000 customers and using their transactions.

In [12]:
N_USERS = 25_000

# Consider only customers with age defined.
customers_df.dropna(inplace=True, subset=["age"])
customer_subset_df = customers_df.sample(N_USERS, random_state=27)
trans_df = trans_df.merge(customer_subset_df["customer_id"])

print(f"Subset has {len(trans_df):,} transactions in total.")

Subset has 585,510 transactions in total.


### Feature Engineering

Next, we do some feature engineering.

The time of the year a purchase was made should be a strong predictor, as seasonality plays a big factor in fashion purchases. Here, we will use the month of the purchase as a feature. Since this is a cyclical feature (January is as close to December as it is to February), we'll map each month to the unit circle using sine and cosine.

In [2]:
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [12]:
%%writefile transformations.py

import numpy as np

def month_sin(t_dat):
    month = t_dat.month - 1
    C = 2*np.pi/12
    return np.sin(month*C).item()

def month_cos(t_dat):
    month = t_dat.month - 1
    C = 2*np.pi/12
    return np.cos(month*C).item()

Writing transformations.py


In [13]:
# create transformation functions for computing sin and cos of month

from transformations import month_sin, month_cos

fns = [fn.name for fn in fs.get_transformation_functions()]

if "month_sin" not in fns:
    month_to_sin = fs.create_transformation_function(month_sin, output_type=float, version=1)
    month_to_sin.save()
    
if "month_cos" not in fns:
    month_cos = fs.create_transformation_function(month_cos, output_type=float, version=1)
    month_cos.save()

We'll also remove columns with null values.

In [16]:
customers_df.dropna(axis=1, inplace=True)
articles_df.dropna(axis=1, inplace=True)

In [17]:
trans_df["month_sin"] = trans_df["t_dat"]
trans_df["month_cos"] = trans_df["t_dat"]

convert python datetime object to unix epoch milliseconds 

In [19]:
import numpy as np
trans_df.t_dat = trans_df.t_dat.values.astype(np.int64) // 10 ** 6

### Feature Groups

A [feature group](https://docs.hopsworks.ai/feature-store-api/latest/generated/feature_group/) can be seen as a collection of conceptually related features.

Before we can create a feature group we need to connect to our feature store.

To create a feature group we need to give it a name and specify a primary key. It is also good to provide a description of the contents of the feature group.

In [20]:
customers_fg = fs.create_feature_group(
    name="customers",
    description="Customers data including age and postal code",
    primary_key=["customer_id"],
    online_enabled=True
)

Here we have also set `online_enabled=True`, which enables low latency access to the data. A full list of arguments can be found in the [documentation](https://docs.hopsworks.ai/feature-store-api/latest/generated/api/feature_store_api/#create_feature_group).

At this point, we have only specified some metadata for the feature group. It does not store any data or even have a schema defined for the data. To make the feature group persistent we populate it with its associated data using the `save` function.

In [21]:
customers_fg.insert(customers_df)

Feature Group created successfully, explore it at 
https://hopsworks0.logicalclocks.com/p/119/fs/67/fg/15


Uploading Dataframe: 0.00% |          | Rows 0/1356119 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: customers_2_offline_fg_backfill
Job started successfully, you can follow the progress at 
https://hopsworks0.logicalclocks.com/p/119/jobs/named/customers_2_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7efd60ffd840>, None)

Let's do the same thing for the rest of the data frames.

In [22]:
articles_fg = fs.create_feature_group(
    name="articles",
    description="Fashion items data including type of item, visual description and category",
    primary_key=["article_id"],
    online_enabled=True
)
articles_fg.insert(articles_df)

Feature Group created successfully, explore it at 
https://hopsworks0.logicalclocks.com/p/119/fs/67/fg/16


Uploading Dataframe: 0.00% |          | Rows 0/105542 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: articles_2_offline_fg_backfill
Job started successfully, you can follow the progress at 
https://hopsworks0.logicalclocks.com/p/119/jobs/named/articles_2_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7efc9b985000>, None)

In [23]:
trans_fg = fs.create_feature_group(
    name="transactions",
    version=1,
    description="Transactions data including customer, item, price, sales channel and transaction date",
    primary_key=["customer_id", "article_id"], 
    online_enabled=True,
    event_time="t_dat"
)
trans_fg.insert(trans_df)

Feature Group created successfully, explore it at 
https://hopsworks0.logicalclocks.com/p/119/fs/67/fg/17


Uploading Dataframe: 0.00% |          | Rows 0/585510 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: transactions_1_offline_fg_backfill
Job started successfully, you can follow the progress at 
https://hopsworks0.logicalclocks.com/p/119/jobs/named/transactions_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7ef99c33ce20>, None)

You should now be able to inspect the feature groups in the Hopsworks UI.

### Feature Views

A [feature view](https://docs.hopsworks.ai/feature-store-api/latest/generated/feature_view/) can be seen as a logical view over a set of features that may come from different feature groups.

Feature views provides an Offline and Online API that can be used to generate training data or retrieve online feature vectors at inference time.

Now we can create two feature views for customers and articles, that will be used during model serving.

In [24]:
customers_query = customers_fg.select_all()
customers_query

In [25]:
customers_feature_view = fs.create_feature_view(
    name='customers',
    query=customers_query
)

Feature view created successfully, explore it at 
https://hopsworks0.logicalclocks.com/p/119/fs/67/fv/customers/version/2


In [26]:
articles_query = articles_fg.select_all()
articles_query

In [27]:
articles_feature_view = fs.create_feature_view(
    name='articles',
    query=articles_query
)

Feature view created successfully, explore it at 
https://hopsworks0.logicalclocks.com/p/119/fs/67/fv/articles/version/2


### Next Steps

In the next notebook we'll create a dataset that we can train a retrieval model on.